## Create a data codebook [optional]
http://www.medicine.mcgill.ca/epidemiology/Joseph/PBelisle/CodebookCookbook.html#:~:text=1%201.%20Introduction%20Writing%20a%20codebook%20is%20an,to%20SAS%20code%20...%208%208.%20Final%20words  

## Available tables

1. Time-series data of fundings in different areas (dollars in millions and rounded)
from 2017 - 2023
(we could get even earlier data from the historical fiscal year reports (.pdf) ) 
https://report.nih.gov/funding/categorical-spending#/ 

OPTIONAL: SCRAPE THE MOST UP-TO=DATE DATA

In [1]:
import pandas as pd
import regex as re

In [2]:
fund_sum = pd.read_excel("RCDCFundingSummary_02172023.xlsx")

In [3]:
fund_sum.head()

,"Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC)","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).1","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).2","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).3","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).4","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).5","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).6","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).7","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).8","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).9",...,"Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).11","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).12","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).13","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).14","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).15","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).16","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).17","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).18","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).19","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).20"
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Research/Disease Areas \n (Dollars in millions...,2008,2009,2009 ARRA,2010,2010 ARRA,2011,2012,2013,2014,...,2016,2017,2018,2019,2020,2021,2022 Estimated,2023 Estimated,2019 US Mortality 19,2019 US Prevalence SE 19
2,Acquired Cognitive Impairment,+,+,+,+,+,+,+,+,+,...,1132,1560,1978,2423,2897,3259,3555,3354,-,-
3,Acute Respiratory Distress Syndrome,82,103,17,110,22,96,98,95,85,...,103,107,123,126,158,148,159,141,10482,-
4,Adolescent Sexual Activity,+,+,+,80,7,69,76,70,68,...,91,99,96,102,88,94,92,94,-,-


In [99]:
proj = pd.read_csv("abl.csv")

2. NIH Awards: by project names, prpoject number (Activity code included) Location, Organization, admin IC, funding mechanisms, direct costs, and indirect costs per fiscal year and project number 
https://report.nih.gov/award/index.cfm?ot=&fy=2023&state=&ic=&fm=&orgid=&distr=&rfa=&om=n&pid=#tab5 

OPTIONAL: SCRAPE THE MOST UP-TO-DATE DATA (MIGHT NEED SELENIUM)

In [49]:
# retrieving location: longitude and latitude based on city and state
from geopy import Nominatim
import numpy as np
geolocator = Nominatim(user_agent="app")
proj["FULL_LOC"] = proj["CITY"] + " " + proj["STATE OR COUNTRY NAME"]
#county code
https://raw.githubusercontent.com/plotly/dash-sample-apps/main/apps/dash-opioid-epidemic/data/lat_lon_counties.csv


In [60]:
import time

In [64]:
# because geocoder is such a pain in the ass 
loc = list(proj["FULL_LOC"].unique())
batch = 30
lat = []
lon = []
geo = []
for i in range(len(loc)//batch):
    print("batch: "+str(i))
    lower = i * batch
    upper = min(len(loc),(i + 1) * batch)
    geo = geo + [geolocator.geocode(i) for i in loc[lower:upper]] 
    lat = lat + [i.latitude if i is not None else np.nan for i in geo]
    lon = lon + [i.longitude if i is not None else np.nan for i in geo]
time.sleep(30)



batch: 0
batch: 1
batch: 2
batch: 3
batch: 4
batch: 5


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=WOODSIDE+CALIFORNIA&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [ ]:

lat_dict = dict(zip(loc,lat))
lon_dict = dict(zip(loc,lon))

lat_dict.update({'WASHINGTON DIST OF COL':38.8950368})
lon_dict.update({'WASHINGTON DIST OF COL': -77.0365427})

In [258]:
cfips = pd.read_csv("county-fips.csv")
cfips["full_name"] = (cfips["county_name"]+" "+cfips["state_name"]).str.upper()
fip_dict = dict(zip(list(cfips["full_name"]),list(cfips["fips"])))
fip_dict


{'AUTAUGA COUNTY ALABAMA': 1001,
 'BALDWIN COUNTY ALABAMA': 1003,
 'BARBOUR COUNTY ALABAMA': 1005,
 'BIBB COUNTY ALABAMA': 1007,
 'BLOUNT COUNTY ALABAMA': 1009,
 'BULLOCK COUNTY ALABAMA': 1011,
 'BUTLER COUNTY ALABAMA': 1013,
 'CALHOUN COUNTY ALABAMA': 1015,
 'CHAMBERS COUNTY ALABAMA': 1017,
 'CHEROKEE COUNTY ALABAMA': 1019,
 'CHILTON COUNTY ALABAMA': 1021,
 'CHOCTAW COUNTY ALABAMA': 1023,
 'CLARKE COUNTY ALABAMA': 1025,
 'CLAY COUNTY ALABAMA': 1027,
 'CLEBURNE COUNTY ALABAMA': 1029,
 'COFFEE COUNTY ALABAMA': 1031,
 'COLBERT COUNTY ALABAMA': 1033,
 'CONECUH COUNTY ALABAMA': 1035,
 'COOSA COUNTY ALABAMA': 1037,
 'COVINGTON COUNTY ALABAMA': 1039,
 'CRENSHAW COUNTY ALABAMA': 1041,
 'CULLMAN COUNTY ALABAMA': 1043,
 'DALE COUNTY ALABAMA': 1045,
 'DALLAS COUNTY ALABAMA': 1047,
 'DEKALB COUNTY ALABAMA': 1049,
 'ELMORE COUNTY ALABAMA': 1051,
 'ESCAMBIA COUNTY ALABAMA': 1053,
 'ETOWAH COUNTY ALABAMA': 1055,
 'FAYETTE COUNTY ALABAMA': 1057,
 'FRANKLIN COUNTY ALABAMA': 1059,
 'GENEVA COUNTY ALABA

In [262]:
# just downloaded the county and city fips code from us census data and mapped
city = pd.read_csv("uscities.csv")
city.city = city.city.str.upper()
city.county_name = city.county_name.str.upper()
city.state_name = city.state_name.str.upper()
city.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,NEW YORK,New York,NY,NEW YORK,36081,QUEENS,40.6943,-73.9249,18972871,10768.2,shape,False,True,America/New_York,1,11229 11226 11225 11224 11223 11221 11220 1138...,1840034016
1,LOS ANGELES,Los Angeles,CA,CALIFORNIA,6037,LOS ANGELES,34.1141,-118.4068,12121244,3267.6,shape,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90035 90034 9003...,1840020491
2,CHICAGO,Chicago,IL,ILLINOIS,17031,COOK,41.8375,-87.6866,8595181,4576.6,shape,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,MIAMI,Miami,FL,FLORIDA,12086,MIAMI-DADE,25.7840,-80.2101,5711945,4945.7,shape,False,True,America/New_York,1,33128 33129 33125 33126 33127 33149 33144 3314...,1840015149
4,DALLAS,Dallas,TX,TEXAS,48113,DALLAS,32.7935,-96.7667,5668165,1522.2,shape,False,True,America/Chicago,1,75287 75098 75234 75254 75251 75252 75253 7503...,1840019440


In [275]:
def geo_encode(dat,city,fip_dict):
    proj = dat.copy()
    #BECAUSE THERE ARE DUPLICATED VALUES AND USUALLY THE FIRST OCCURENCES ARE RIGHT
    citys = list(city.city)
    lngs =  list(city.lng)
    lats = list(city.lat)
    fips = list(city.county_fips)
    countys = list(city.county_name)
    
    lon_dict = dict()
    lat_dict = dict()
    ct_dict = dict()
    fip_dict = dict()
    [lon_dict.update({i:lngs[citys.index(i)]}) for i in set(citys)]
    [lat_dict.update({i:lats[citys.index(i)]}) for i in set(citys)]
    [ct_dict.update({i:countys[citys.index(i)]}) for i in set(citys)]
    [fip_dict.update({i:fips[citys.index(i)]}) for i in set(citys)]

    proj["LON"] = proj["CITY"].map(lon_dict)
    proj["LAT"] = proj["CITY"].map(lat_dict)
    proj["COUNTY"] = proj["CITY"].map(ct_dict)
    proj["FIPS"] = proj["CITY"].map(fip_dict)
    
    notfound = list(proj[proj["LON"].isna()]["CITY"] + " "\
                    +proj[proj["LON"].isna()]['STATE OR COUNTRY NAME']).unique()
    #print(notfound)

    geo = [geolocator.geocode(i) for i in notfound] 
    lat = [i.latitude if i is not None else np.nan for i in geo]
    lon = [i.longitude if i is not None else np.nan for i in geo]
    county = [i.address.upper().split(", ")[1] if i is not None else np.nan for i in geo]
    fip = [fip_dict[i.address.upper().split(", ")[1] + " " +i.address.upper().split(", ")[2]] \
           if i is not None else np.nan 
           for i in geo]
    
    notfoundl = list(notfound)
    lat_add = dict(zip(notfoundl,lat))
    lon_add = dict(zip(notfoundl,lon))
    ct_add = dict(zip(notfoundl,county))
    fip_add = dict(zip(notfoundl,fip))

    ct_dict.update(ct_add)
    fip_dict.update(fip_add)
    lat_dict.update(lat_add)
    lon_dict.update(lon_add)
    
    proj["COUNTY"] = proj["CITY"].map(ct_dict)
    proj["FIPS"] = proj["CITY"].map(fip_dict)
    proj["LON"] = proj["CITY"].map(lon_dict)
    proj["LAT"] = proj["CITY"].map(lat_dict)
    proj["LON"] = proj["LON"].apply(lambda x: np.nan if re.findall(r"[-\d\.]+",str(x))[0] is None else re.findall(r"[-\d\.]+",str(x))[0])
    proj["LAT"] = proj["LAT"].apply(lambda x: np.nan if re.findall(r"[-\d\.]+",str(x))[0] is None else re.findall(r"[-\d\.]+",str(x))[0])
    return proj 



In [274]:
geolocator.geocode("PARKTOWN, US").address.upper().split(", ")

['PARKTOWN', 'WARREN COUNTY', 'NORTH CAROLINA', 'UNITED STATES']

In [270]:
proj["CITY"] = proj["CITY"].str.upper()
proj["STATE OR COUNTRY NAME"] = proj['STATE OR COUNTRY NAME'].str.upper()
proj = geo_encode(proj,city,fip_dict)

KeyError: 'STATE'

In [169]:
proj["MONTH"] = pd.to_datetime(proj["AWARD NOTICE DATE"]).dt.month
proj["YEAR"] = pd.to_datetime(proj["AWARD NOTICE DATE"]).dt.year

In [224]:
proj = proj[proj['FUNDING MECHANISM'].notna()]

In [225]:
proj.columns

Index(['ORGANIZATION NAME', 'ORGANIZATION ID (IPF)', 'PROJECT NUMBER',
       'FUNDING MECHANISM', 'NIH REFERENCE', 'PI NAME', 'PI PERSON ID',
       'PROJECT TITLE', 'DEPT NAME', 'NIH DEPT COMBINING NAME',
       'NIH MC COMBINING NAME', 'DIRECT COST', 'INDIRECT COST', 'FUNDING',
       'CONGRESSIONAL DISTRICT', 'CITY', 'STATE OR COUNTRY NAME', 'ZIP CODE',
       'ATTRIBUTED TO MEDICAL SCHOOL', 'MEDICAL SCHOOL LOCATION',
       'INSTITUTION TYPE', 'AWARD NOTICE DATE', 'FOA', 'MONTH', 'YEAR', 'LON',
       'LAT'],
      dtype='object')

In [226]:
proj.to_csv("abl_convert.csv")

3. Activity code: categories of fundigns with titles, descriptions, and activity code 
NIH uses activity codes (e.g. R01, R43, etc.) to differentiate the wide variety of research-related programs we support. NIH Institutes and Centers (ICs) Link to External Site may vary in the way they use activity codes; not all ICs accept applications for all types of grant programs or they apply specialized eligibility criteria. Look closely at Funding Opportunity Announcements (FOAs) to determine which ICs participate and the specifics of eligibility.
https://grants.nih.gov/grants/funding/ac_search_results.htm



In [19]:
codes = pd.read_excel("ActivityCodes.xls")

In [20]:
codes.head()

,ID,ACT_CATEGORY,ACT_CODE,ACT_TITLE,ACT_DESCRIPTION,ACT_WEB_ADDRESS
0,117,Research Construction Programs,C06,Research Facilities Construction Grant,"To provide matching Federal funds, up to 75%, ...",NaN
1,1,Institutional Training and Director Program Pr...,D43,International Research Training Grants,To support research training programs for US a...,http://grants.nih.gov/training/F_files_non_nrs...
2,2,Institutional Training and Director Program Pr...,D71,International Research Training Planning Grant,To plan for the preparation of an application ...,http://grants.nih.gov/training/F_files_non_nrs...
3,3,Institutional Training and Director Program Pr...,DP1,NIH Director’s Pioneer Award (NDPA),To support individuals who have the potential ...,http://commonfund.nih.gov/pioneer/
4,4,Institutional Training and Director Program Pr...,DP2,NIH Director’s New Innovator Awards,To support highly innovative research projects...,http://commonfund.nih.gov/newinnovator/


4. API  
https://api.reporter.nih.gov/

will get a holistic dataset on projects through API. Submit query and obtain json。
However, there is an entry limit of 500 imposed on each query. 

In [31]:
import requests

headers = {
    'accept': 'application/json',
    # Already added when you pass json=
    # 'Content-Type': 'application/json',
}

json_data = {
    'searchId': '*',
    'criteria': {
        'useRelevance': True,
        'includeActiveProjects': True,
        'offset': 0,
        'limit': 3
    },
}

response = requests.post('https://api.reporter.nih.gov/v2/projects/search', headers=headers, json=json_data)

In [34]:
import pandas as pd
import json
dict = json.loads(response.text)
df = pd.json_normalize(dict['results'])
df.head()
       

,appl_id,subproject_id,fiscal_year,project_num,project_serial_num,award_type,activity_code,award_amount,is_active,project_num_split,...,full_study_section.irg_code,full_study_section.srg_code,full_study_section.srg_flex,full_study_section.sra_designator_code,full_study_section.sra_flex_code,full_study_section.group_code,full_study_section.name,full_study_section.url,full_study_section.cmte_id,full_study_section.cluster_irg_code
0,9795459,None,2018,261201500036I-0-26100009-2,None,None,N01,930464.0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10324021,None,2021,1R43OH012283-01,OH012283,1,R43,243241.0,False,NaN,...,ZRG1,ZRG1,,AIDC,V,13,Special Emphasis Panel[ZRG1 AIDC-V (13)],NaN,NaN,NaN
2,10294664,None,2020,75N92020P00309-0-0-3,None,None,N02,100000.0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10287968,None,2020,75N95019D00013-0-759502000001-1,None,None,N01,2113098.0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10329701,None,2021,75N95019D00013-P00002-759502000001-1,None,None,N01,549285.0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
